# 1: The Pythonic Style (Foundations)

## Comma Placement
- **put a comma after every entry especially the last one** in your list, dict and set 
- helps to catch hard-to-spot bugs
- helps with git diff: since most source control systems are line-based and have a hard time highlighting multiple changes to a single line.
- if you forget to set a comma, strings get glued togethter even if they are in different lines - it's called 'string literal concatenation'

In [ ]:
names = [ 
    'Alice', 
    'Bob',
    'Dilbert', 
    ]

## Underscores, Dunders, and More

In Python, underscores are more than just separators; they dictate how the interpreter and other programmers interact with your code.

<details><summary>Naming Conventions</summary>
- **Single Leading Underscore (Internal Use Hint) `_var`**: Naming convention indicating a name is meant for internal use. It is a "gentle hint" to other programmers. It is not enforced by the interpreter, except that it prevents the variable from being imported when using from module import *.

- **Single Trailing Underscore (Conflict Resolution) `var_`**: Used to avoid naming conflicts with Python keywords (like class, list, or type). Appending a single trailing underscore allows you to use a reserved word as a variable name without a syntax error. 

- **Double Leading Underscore (Name Mangling) `__var`**: Triggers "Name Mangling" in a class context to protect attributes from being accidentally overridden by subclasses. The interpreter renames the variable to _ClassName__var internally. This is the closest Python comes to "private" variables.

- **Double Leading and Trailing Underscore (Dunders) `__var__`**: Double Under" (Dunder) methods are reserved for special Python protocols (e.g., operator overloading, lifecycle management). The interpreter calls these automatically in specific contexts (like len() calling __len__). Never invent your own dunder names; only use the ones defined by the language.
</details>

In [ ]:
class MyClass:
    def __init__(self):
        self._internal_id = 123  # Hint: Don't touch this outside the class


obj = MyClass()
print(obj._internal_id)  # Works, but violates convention.


# 'class' is a keyword, so we use 'class_'
def create_element(name, class_="primary-btn"):
    return f"<{name} class='{class_}'>"

class Parent:
    def __init__(self):
        self.__secret = "Parent Data"


class Child(Parent):
    def __init__(self):
        super().__init__()
        self.__secret = "Child Data"  # Does NOT overwrite Parent's __secret


p = Parent()
c = Child()

# Direct access to __secret fails
# print(p.__secret) -> AttributeError

# Access is possible via the mangled name:
print(p._Parent__secret)
print(c._Child__secret)



class Book:
    def __init__(self, title):
        self.title = title

    def __repr__(self):
        return f"Book('{self.title}')"


my_book = Book("Advanced Python")
print(my_book)  # Automatically triggers __repr__



# 1. Ignoring values during unpacking
for _, value in [("A", 1), ("B", 2)]:
    print(value)

# 2. Simple repetition where the index doesn't matter
for _ in range(3):
    print("Hello!")

# 2: Resource Management & String Handling

### Template string
- need to be imported
- no string formatting allowed
- When to use: 
  -  When handling format strings generated by users of your program. 
  -  Due to their reduced complexity,template strings are a safer choice.
  -  More complex formatting might introduce security vulnerabilities. 

In [ ]:
from string import Template
t = Template('Hey, $name!')
t.substitute(name=name)


templ_string = 'Hey $name, there is a $error error!'
Template(templ_string).substitute(name=name, error=hex(errno)) # formatting need to be done beforehand

'Hey, Bob!'

In [ ]:
'''
SECURITY RISK: it's possible for format strings to access arbitrary variables in your program.
A malicious user can supply a format string they can also potentially leak secret keys and 
other sensitive information! E.g. an attacker can extract our secret string by accessing the 
__globals__ dictionary from the format string.
'''

SECRET = 'this-is-a-secret'

class Error:
    def __init__(self):
        pass

err = Error()
user_input = '{error.__init__.__globals__[SECRET]}'

user_input.format(error=err)

'this-is-a-secret'

In [ ]:
user_input = '${error.__init__.__globals__[SECRET]}'
Template(user_input).substitute(error=err)

ValueError: Invalid placeholder in string: line 1, col 1

# 3: Functional Programming & Modern Control Flow

## Functions
- Everything in Python is an object, including functions. You can assign them to variables, store them in data structures, and pass or return them to and from other functions (ﬁrst-class functions.)
- First-class functions allow you to abstract away and pass around behavior in your programs.
- Functions can be nested and they can capture and carry some of the parent function’s state with them. Functions that do this are called closures.
- Objects can be made callable. In many cases this allows you to treat them like functions.

#### Functions as First-Class Citizens
- Functions can be stored in data structures#e.g. in lists ``funcs = [fct1, fct2, fct3]``
<details><summary>Functions can be passed to other functions</summary>
- It allows you to abstract away and pass around behavior in your programs. 
- Here, the greet function stays the same but you can pass in diﬀerent behaviors.
- Functions that can accept other functions as arguments are also called higher-order functions. 
- Higher-order functions are a necessity for the functional programming style.
- The classical example for higher-order functions in Python is the built-in map function. It takes a function object and an iterable, and then calls the function on each element in the iterable, yielding the results as it goes along.
</details>

In [22]:
# no parentheses
funcs = [str.lower, str.upper, str.capitalize]
for f in funcs:
    print(f, f('hey there'))

funcs[0]('heyyo')



def greet(func):
    greeting = func('Hi, I am a Python program')
    print(greeting)

greet(str.upper)


# map  goes through the list and applies the 'capitalized' fct
list(map(str.capitalize, ["hello", "hey", "hi"]))

<method 'lower' of 'str' objects> hey there
<method 'upper' of 'str' objects> HEY THERE
<method 'capitalize' of 'str' objects> Hey there
HI, I AM A PYTHON PROGRAM


['Hello', 'Hey', 'Hi']

## Objects can behave like functions
- While all functions are objects in Python, the reverse isn’t true. Objects aren’t functions. But they can be made callable, which allows you to treat them like functions in many cases.
- If an object is callable it means you can use the round parentheses function call syntax on it and even pass in function call arguments. All made possible by the ``__call__`` dunder method.
- Behind the scenes, “calling” an object instance as a function attempts to execute the object’s ``__call__`` method. There’s a built-in callable function to check whether an object appears to be callable or not.

In [93]:
class Adder:
    def __init__(self, n):
        self.n = n

    def __call__(self, x):
        return self.n + x
    
# deﬁning a callable object
plus_3 = Adder(3)
# plus_3 is now callable
plus_3(4)

callable('hello') # 'hello' is not callable
callable(plus_3)

True

## Functions can capture local state
- inner functions can capture and carry the parents functions state
- Inner functions that access parameters deﬁned in the parent function are called lexical closures (or just closures). 
- A closure remembers the values from its enclosing lexical scope even when the program flow is no longer in that scope.

In [ ]:
# here inner functions do not have the text parameter
# but they can 'capture' it from the parent function


def get_speak_func(text, volume):
    def whisper():
        return text.lower() + '...'
    
    def yell():
        return text.upper() + '!'
    
    if volume > 0.5:
        return yell
    else:
        return whisper
    
get_speak_func('Hello, World', 0.7)()

'HELLO, WORLD!'

In [ ]:
def make_adder(n):
    def add(x):
        return x + n
    return add

# make_adder is the enclosing scope
plus_3 = make_adder(3) # n is set to 3
plus_5 = make_adder(5) # n is et to 5

seven = plus_3(4) # x is set to 4
nine = plus_5(4) # x is set to 4

print(seven) 
print(nine)

7
9


##  Decorators
- Add functionality to an existing callable. They wrap functions into other functions, without permanently modifying the the wrapped functions. 
- **A decorator should return a function**. <br>
  Here, uppercase() returns the wrapper() function, which will replace the decorated function greet. 
- The ``@ syntax`` is just a shorthand for calling the decorator on an input function.
- You can **stack multiple decorators** on a single function decorators will be applied from bottom to top. 
- When you define the inner function, it creates a **closure** over the decorated function argument. This means that the inner function has access to the decorated function variable even after the outer function has finished.
- The purpose of the inner function wrapper() is to control the execution of the original function. In this example, it calls func() and modifies its return value by converting it to uppercase.
- As a **debugging** best practice, use the ``functools.wraps`` helper in your own decorators to carry over metadata from the undecorated callable to the decorated one.

In [109]:
# uppercase() wraps the given function in another function.
def uppercase(func):
    def wrapper(): # closure/wrapper function
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    # A decorator should return a function. 
    return wrapper

# greet() is decorated with uppercase().
@uppercase
def greet():
    return 'Hello!'

greet()

'HELLO!'

## Stacking Multiple Decorators

In [111]:
# you can stack multiple decorators to a function

def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

# decorators are ordered from bottom to top - stacked
@strong
@emphasis
def greet():
    return 'Hello!'

greet()

'<strong><em>Hello!</em></strong>'

## Decorators with Arguments
-  * and ** operators collect all positional and keyword arguments and stores them in variables (args and kwargs)
-  The wrapper forwards the collected arguments to the original input function using the * and ** “argument unpacking” operators.

In [112]:
# Decorating a function with trace and then calling it will print the 
# arguments passed to the decorated function and its return value.

def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}() '
            f'with {args}, {kwargs}')
        
        original_result = func(*args, **kwargs)

        print(f'TRACE: {func.__name__}() '
            f'returned {original_result!r}')
        
        return original_result
    
    return wrapper



@trace
def say(name, line):
    return f'{name}: {line}'

say('Jane', 'Hello, World')

TRACE: calling say() with ('Jane', 'Hello, World'), {}
TRACE: say() returned 'Jane: Hello, World'


'Jane: Hello, World'

## Debuggable Decorators
- When you use a decorator, you’re  replacing one function with another. One downside of this process is that it “hides” some of the metadata attached to the original (undecorated) function.
- The docstring, and parameter list of the original (undecorated) function are hidden by the wrapper closure.
- ``functools.wraps`` decorator ﬁxes this, it copies over the lost metadata from the undecorated function to the decorator closure.
- As a best practice, use functools.wraps in all of the decorators.

In [116]:
import functools

def uppercase(func):
    # without this line we could not call the 
    # __name__ or __doc__ of the decorated function
    @functools.wraps(func) 
    def wrapper():
        return func().upper()   
    return wrapper

@uppercase
def greet():
    """Return a friendly greeting."""
    return 'Hello!'


greet.__name__
greet.__doc__

'Return a friendly greeting.'

## Sequence Unpacking
Extracting elements from an iterable (list, tuple, string) into variables. Using * to capture "everything else" into a list. This is much cleaner than using manual slicing.

In [28]:
# Standard Unpacking
data = ["Alice", 30, "Engineer"]
name, age, job = data

# Extended Unpacking (The "Rest" variable)
numbers = [1, 2, 3, 4, 5, 6]
first, *middle, last = numbers

print(first)  # 1
print(middle)  # [2, 3, 4, 5] (Always a list)
print(last)  # 6


1
[2, 3, 4, 5]
6


## Deep Unpacking & Merging
Using asterisks to "explode" an iterable or dictionary into a new structure.

`*` works for sequences (lists/tuples), while `**` works for dictionaries.

In [ ]:
# Merging Lists
list_a = [1, 2]
list_b = [3, 4]
combined_list = [*list_a, *list_b]  # [1, 2, 3, 4]

# Merging Dictionaries (Python 3.5+)
dict_a = {"x": 1, "y": 2}
dict_b = {"y": 3, "z": 4}

# Note: dict_b will overwrite shared keys (y)
merged_dict = {**dict_a, **dict_b}  # {'x': 1, 'y': 3, 'z': 4}

{*dict_a} # keys


{'x', 'y'}

## Function Argument Packing (*args and **kwargs)

Allowing a function to accept any number of positional or keyword arguments.

Inside the function, args becomes a tuple of all positional arguments, and kwargs becomes a dictionary of all keyword arguments.

In [37]:
def flexible_function(*args, **kwargs):
    print(f"Positional (tuple): {args}")
    print(f"Keywords (dict): {kwargs}")


flexible_function(1, 2, 3, name="Alice", state="Active")


Positional (tuple): (1, 2, 3)
Keywords (dict): {'name': 'Alice', 'state': 'Active'}


## Argument Forwarding (The Wrapper Pattern)

Passing all received arguments directly to another function. This is the foundation of Decorators.

You pack the arguments in the definition and unpack them in the call.

In [39]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with {args} {kwargs}")
        return func(*args, **kwargs)

    return wrapper


@trace
def greet(name, greeting="Hello"):
    return f"{greeting}, {name}!"


print(greet("Bob", greeting="Hi"))

Calling greet with ('Bob',) {'greeting': 'Hi'}
Hi, Bob!


## Mandatory Keyword-Only Arguments

Forcing the caller to use named arguments for clarity.

Any argument placed after a bare * in a function signature must be passed as a keyword.

In [42]:
# 'as_pdf' MUST be named when calling
def generate_report(data, *, as_pdf=False):
    pass

# generate_report(my_data, True)  # Fails: TypeError
generate_report(my_data, as_pdf=True)  # Works

NameError: name 'my_data' is not defined

## Comprehensions

### Comprehensions Syntax

`[expression for item in iterable]` 

`[expression for item in iterable if condition]` 

In [ ]:
# without condition
[num for num in range(1, 12)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [ ]:
# with condition
[num**2 for num in range(1, 12) if num % 2 == 0]

[4, 16, 36, 64, 100]

### Nested List Comprehension

In [ ]:
[[c for c in range(5)] for r in range(5)]

[[0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4]]


### dictionary comprehension

In [ ]:
ctemps = [0, 12, 34, 100]

# braces and key: value syntax 
tempDict = {
    t: (t * 9 / 5) + 32 for t in ctemps if t < 100
}  

print(tempDict)
print(tempDict[12])  # fetch value of dict with key=12

{0: 32.0, 12: 53.6, 34: 93.2}
53.6


In [ ]:
team1 = {"Jones": 24, "Jameson": 18, "Smith": 58, "Burns": 7}
team2 = {"White": 12, "Macke": 88, "Pierce": 4}

# do not get more complex in a comprehension
newTeam = {k: v 
           for team in (team1, team2) 
           for k, v in team.items()}

print(newTeam)

{'Jones': 24, 'Jameson': 18, 'Smith': 58, 'Burns': 7, 'White': 12, 'Macke': 88, 'Pierce': 4}


### set comprehension

In [ ]:
ctemps = [0, 10, 12, 14, 10, 23, 12, 34, 34, 100, 100]
ftemps1 = [(t * 9 / 5) for t in ctemps]  # this is a list

# when we do not use kev-value pairs
# curly brackets create a set, sets do not allow duplicates
ftemps2 = {(t * 9 / 5) for t in ctemps}
print(ftemps1)
print(ftemps2)

[0.0, 18.0, 21.6, 25.2, 18.0, 41.4, 21.6, 61.2, 61.2, 180.0, 180.0]
{0.0, 41.4, 18.0, 180.0, 21.6, 25.2, 61.2}


## Match ... Case

In [ ]:
def check_number(x):
    match x:
        case 10:
            print("It's 10")
        case 20:
            print("It's 20")
        case _:
            print("It's neither 10 nor 20")


check_number(10)
check_number(30)



def num_check(x):
    match x:
        case 10 | 20 | 30:  # Matches 10, 20, or 30
            print(f"Matched: {x}")
        case _:
            print("No match found")


num_check(10)
num_check(20)
num_check(25)

It's 10
It's neither 10 nor 20
Matched: 10
Matched: 20
No match found


# 4: Object-Oriented Programming (Deep Dive)

### Abstract Base Classes (ABCs)
An abstract base class is a class that cannot be instantiated on its own. It serves as a blueprint for other classes, defining a set of methods and attributes that subclasses must implement. ABCs are useful for creating a common interface for a group of related classes, making the code more organized and easier to maintain.

An abstract method is a method that is declared, but contains no implementation.

@abstractmethod: Enforces that subclasses implement the abstract method.

When to Use ABCs

- Standardize interfaces (e.g., data loaders, parsers, plugins).
- Enforce contracts in large codebases or libraries.
- Document expectations for subclasses.


In [11]:
from abc import ABC, abstractmethod

# define an abstract base class
class AbstractClassExample(ABC):
    def __init__(self, value):
        self.value = value
        super().__init__()
        
    # we are required to implement the abstract method in the child class
    @abstractmethod
    def do_something(self):
        pass


# A class that is derived from an abstract class cannot be instantiated unless all of its abstract methods are overridden.
class DoAdd42(AbstractClassExample):
    def do_something(self):
        return self.value + 42


class DoMul42(AbstractClassExample):
    def do_something(self):
        return self.value * 42


x = DoAdd42(10)
y = DoMul42(10)

print(x.do_something())
print(y.do_something())

52
420


### Name Mangling: __var

"Protect this attribute from being accidentally overridden in subclasses."

The interpreter rewrites the name to _ClassName__var to make it harder to access from outside, effectively providing a level of "private" scoping.

In [ ]:
class Parent:
    def __init__(self):
        self.__private = "Parent secret"


class Child(Parent):
    def __init__(self):
        super().__init__()
        self.__private = "Child secret"


p = Parent()
# print(p.__private)  # Fails with AttributeError
print(p._Parent__private)  # Accessible via "mangled" name


## Double leading and trailing Underscores __var__:<br>
No name-mangeling happens here. Names that have both leading and trailing double underscores are reserved for special use in the language - **dunder methods** like ``__init__`` & ``__call__``. As far as naming conventions go, it’s best to stay away from using names that start and end with double underscores in your own programs to avoid collisions with future changes to the Python language.

In [ ]:

class PrefixPostfixTest:
    def __init__(self):
        # i tis possible to give names like __bam__ but not recommended
        self.__bam__ = 42

PrefixPostfixTest().__bam__

42

In [ ]:
from collections import namedtuple

# Liskov substiution principle:
# states that you should program in a way where you can substitute
# a subclass for  it's parent witout breaking the program

# namedtuple is a tuple
Point = namedtuple('Point', ['x', 'y'])  
# so Point class is a tuple, but it is a subclass of the built-in tuple
p = Point(1, 2)  


# Liskov substiution violation: 
# namedtuple is not equal to a tuple
if type(p) == tuple:  # Liskov substiution violation
    print("it's a tuple")
else:
    print("it's not a tuple")
    print(f"it's a: {type(p)}")

# check if is instance of tuple, namedtuple is an instance of a tuple (base class)
if isinstance(p, tuple):
    print("it's a tuple")
else:
    print("it's not a tuple")

it's not a tuple
it's a: <class '__main__.Point'>
it's a tuple


# 5: Advanced Data Structures

# 6: System & Diagnostic Tools

## logging module
- levels indicating the severity of events: DEBUG, INFO, WARNING, ERROR, CRITICAL
- for logging to a file rather than the console, filename and filemode can be used
- you can decide the format of the message using format

In [ ]:
import logging

# set up logging in main function
# format your error meassages
level = logging.DEBUG
fmt = '[%(levelname)s] %(asctime)s - %(message)s'
logging.basicConfig(level=level, format=fmt)

# this shows level, name, and message separated by a colon (:)
# debug() and info() messages didn’t get logged. 
# bc. the logging module logs the messages with a severity level of WARNING or above
logging.debug('This is a debug message')
logging.info('This is an info message')
logging.warning('This is a warning message')
logging.error('This is an error message')
logging.critical('This is a critical message')

ERROR:root:This is an error message
CRITICAL:root:This is a critical message


- Be aware that, basicConfig() can only be called once.
- debug(), info(), warning(), error(), and critical() also call basicConfig() without arguments <br>
automatically if it has not been called before.

In [ ]:
# you can set what level of log messages you want to record
import logging

# Remove all handlers associated with the root logger object.
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(level=logging.DEBUG)
logging.debug('This will get logged')

DEBUG:root:This will get logged


- The filemode is set to w, which means the log file is opened in “write mode” 
each time basicConfig() is called, and each run of the program will rewrite the file. 
- The default configuration for filemode is a, which is append.

In [ ]:
import logging

# Remove all handlers associated with the root logger object.
[logging.root.removeHandler(handler) for handler in logging.root.handlers[:]]

logging.basicConfig(
    filename='app.log',
    level=logging.INFO,
    filemode='w',
    format='%(process)d - %(name)s - %(levelname)s - %(message)s',
    force = True)

logging.info('This will get logged to a file')

In [ ]:
# set up logging in main function

# Remove all handlers associated with the root logger object.
[logging.root.removeHandler(handler) for handler in logging.root.handlers[:]]

level = logging.DEBUG
fmt = '%(process)d - %(name)s - [%(levelname)s] %(asctime)s - %(message)s'  # format your error meassages
logging.basicConfig(level=level, format=fmt)

# wherever
logging.debug("debug info")
logging.info("just some info")
logging.error("uh oh :(")

2770 - root - [DEBUG] 2023-03-10 00:02:55,650 - debug info
2770 - root - [INFO] 2023-03-10 00:02:55,652 - just some info
2770 - root - [ERROR] 2023-03-10 00:02:55,653 - uh oh :(


## subprocesses 
- subprocess should be used for accessing system commands
- Python’s subprocess library is designed to launch processes to run external programs,
regardless of the languages used to write them. It interacts with the operating system and can issue shell commands like ls or dir.
- A **process** is the operating system’s abstraction of a running program.
- **Subprocess** is the child of a (Python) process 
- Python subprocess module should be used for accessing system commands, <br>
it's an alternative to using the os module
- Most of your interaction with the Python subprocess module will be via the **run()** function, <br>
while all functions in the subprocess module are convenience wrappers around the **Popen()** <br>
constructor and its instance methods.

In [ ]:
# You can call any application that you can with the Start menu or app bar,
# as long as you know the precise name or path of the program
import subprocess

# subprocess.run('ls')
subprocess.run(["nano"])



def subprocess_with_shell_true():
    # shell= True causes security problems leave it out
    # subprocess.run(["ls -l"], capture_output=True, shell=True)
    subprocess.run(["ls", "-l"], capture_output=True)


subprocess_with_shell_true()

In [ ]:
import subprocess
# subprocess commands are tokenized by putting them in a list
# p1 = subprocess.run(["ls", "-l"])
# print(p1.returncode) # shows errors, 0 means zero errors
# print(p1.args) # passes arguments
# print(p1.stdout) # None, means nothing was captured

# # now stdout is captured in the varibale and not printed in console
# # text=True decodes the byte output to string
# p2 = subprocess.run(["ls", "-l"], capture_output=True, text=True)
# print(p2.stdout)

# # write stdout to a file
# with open('output.txt', 'w') as f:
#     p2 = subprocess.run(["ls", "-l"], stdout=f, text=True)

# # In cas of an error python will NOT throw an error message
# # but we can see that the returncode is not equal to zero
# p2 = subprocess.run(["ls", "-l", 'dne'], capture_output=True, text=True)
# print(p2.returncode)  # see the errorcode
# print(p2.stderr)  # see the error

# # you can write smt like this in your code
# # to check if your subprocess failed
# if p2.returncode != 0: print('You have an error sir')

# # if you want python to throw an error we can pass the argument
# # check=True
# p2 = subprocess.run(["ls", "-l", 'dne'],
#                     capture_output=True,
#                     text=True,
#                     check=True)

# # we can IGNORE ERROR altogether by sending them to dev_null
# p2 = subprocess.run(["ls", "-l", 'dne'],
#                     stderr=subprocess.DEVNULL)

# # feding the output of on process to another process as input
# p3 = subprocess.run(["cat", "test.txt"], capture_output=True, text=True)
# # print(p3.stdout)

# # we search for the word 'test' in the output of p3
# p4 = subprocess.run(["grep", "-n", 'test'],
#                     capture_output=True,
#                     text=True,
#                     input=p3.stdout)  # p3.stdout=p4.input

# print(p4.stdout)

# shell=True makes it possible to write a shell command just like in the console
# shell= True causes SECURITY PROBLEMS leave it out if possible
p3 = subprocess.run(' cat test.txt | grep -n test', # command as a regular string one-liner
                    capture_output=True,
                    text=True,
                    shell=True)


4:test



# 7: Internals & Performance: The GIL & Concurrency

### use numpy

In [ ]:
import numpy as np


def not_using_numpy_pandas():
    x = list(range(100))
    y = list(range(100))
    s = [a + b for a, b in zip(x, y)]

    # better (faster)
    x = np.arange(100)
    y = np.arange(100)
    s = x + y

### Global Interpreter Lock (GIL)
In Python, GIL is a mutex (or a lock) that allows only one thread to execute Python bytecode at a time, even on machines with multiple cores.

The multiprocessing module allows a Python program to spawn independent processes rather than threads. Each process has its own memory space and its own Python interpreter and therefore, each has its own GIL. This allows Python to achieve true parallelism, especially on multi-core machines. 

In [15]:
from multiprocessing import Process


def compute():
    total = 0
    for _ in range(10**7):
        total += 1


p1 = Process(target=compute)
p2 = Process(target=compute)

p1.start()
p2.start()

p1.join()
p2.join()


Traceback (most recent call last):
  File "/home/mz/.pyenv/versions/3.14.2/lib/python3.14/multiprocessing/forkserver.py", line 340, in main
    code = _serve_one(child_r, fds,
                      unused_fds,
                      old_handlers)
  File "/home/mz/.pyenv/versions/3.14.2/lib/python3.14/multiprocessing/forkserver.py", line 380, in _serve_one
    code = spawn._main(child_r, parent_sentinel)
Traceback (most recent call last):
  File "/home/mz/.pyenv/versions/3.14.2/lib/python3.14/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: module '__main__' has no attribute 'compute'
  File "/home/mz/.pyenv/versions/3.14.2/lib/python3.14/multiprocessing/forkserver.py", line 340, in main
    code = _serve_one(child_r, fds,
                      unused_fds,
                      old_handlers)
  File "/home/mz/.pyenv/versions/3.14.2/lib/python3.14/multiprocessing/forkserver.py", line 380, in _serve_one
    code = spawn._main(child_

# Replaces use of default mutable arguments 

Default arguments in Python are evaluated only once. The evaluation happens when the function is defined, instead of every time the function is called. This can inadvertently create hidden shared state, if you use a mutable default argument and mutate it at some point. This means that the mutated argument is now the default for all future calls to the function as well. This is usually unintended behaviour, though it can be useful in limited circumstances for writing  
caches.

In [52]:
# We pass an empty list as the default argument for the emp_list
# bc. we want the the lsit to start from scratch when only a employee names is passed
# but the second tim the fct is called WITHOUT passing an emp_list the old entry still is in the list
# this is bc. default arguments are evaluated once in Python when the fct is created

def add_new_employee(employee, emp_list=[]):
    emp_list.append(employee)
    print(emp_list)


add_new_employee('Stephan')
add_new_employee( 'Gisela' )  # the 'Hello' from the previous fct call still lingers in the list

['Stephan']
['Stephan', 'Gisela']


In [60]:
# if you want mutable default, first set it to None and then assign value in the fct.
def add_new_employee(employee, emp_list=None):
    if emp_list is None: emp_list = []
    emp_list.append(employee)
    return emp_list


emp_list1 =add_new_employee('Stephan')
print(emp_list1)

emp_list2 = add_new_employee('Gisela')
print(emp_list2)

emp_list = add_new_employee('Gisela')
add_new_employee('Gustav', emp_list)

['Stephan']
['Gisela']


['Gisela', 'Gustav']

# Identity (is) vs  Equality (==) 

There is a simple rule of thumb to tell you when to use **==** or **is**.<br>

**==** is for **value equality** (same value). <br>
Use it when you would like to know if two objects have the same value.<br>
is will return True if two variables point to the same object (in memory), <br>
> The operators <, >, ==, >=, <=, and != compare the values of two objects.

**is** is for **object identity**. <br>
Use it when you would like to know if two references refer to the same object.<br>
is will return True if two variables point to the same object (in memory)<br>
Object identity is determined using the id() function.<br>
In Python the None object is a singleton, so it is correct to use *is* when comparing to it.

> Thus, the check for identity is the same as checking for the equality of the IDs of the objects. That is, <br>
>a is b<br>
>is the same as:<br>
>id(a) == id(b)<br>



In [15]:
def  equal_or_identical(x):
    print(x == None)
    print(x == True)
    print(x == False)

    # better
    print(x is None)
    print(x is True)
    print(x is False)


equal_or_identical(False)

False
False
True
False
False
True


In [19]:
def checking_bool_or_len(x):
    print(bool(x))  # bool is True if not False 
    print(len(x) != 0) # len is True if x is not empty

    # Both can be substitued with a plain if x
    # usually equivalent to
    if x: print("Not False and not empty")

checking_bool_or_len("hello")

True
True
Not False and not empty


# iterating over the list directly

In [20]:
a = [1, 2, 3]

# # this is unnecessary
# for i in range(len(a)):
#     v = a[i]
#     print(v)

# instead go over the values directly
for v in a:
    print(v)

# or if you wanted the index
for i, v in enumerate(a):
    print(i, v)

1
2
3
0 1
1 2
2 3


# getpass() to hide input

In [ ]:
from getpass import getpass

username = input('Username: ')
# instead of input('Passwort: ') put getpass(...) and the password 
# is hidden in the console while writing
password = getpass('Password: ')
print('Logging In...')

# python -m
- run a module that is NOT in the cd
- this runs the smtpd module, everything after smtpd are the args of that module 

`python -m smtpd DebuggingServer -n localhost:1025`

to find out about the args of a module run: 
**help(module_name)**

if you justa want attributed and method name of a module check: **dir(module_name)**

when you check a medule with dir() and want to know if smt is an attribute or a method you can: **modulename.method_name** without the () and get infos

In [ ]:
from datetime import datetime
print(help(datetime))

In [ ]:
print(dir(datetime))

['__add__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__rsub__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', 'astimezone', 'combine', 'ctime', 'date', 'day', 'dst', 'fold', 'fromisocalendar', 'fromisoformat', 'fromordinal', 'fromtimestamp', 'hour', 'isocalendar', 'isoformat', 'isoweekday', 'max', 'microsecond', 'min', 'minute', 'month', 'now', 'replace', 'resolution', 'second', 'strftime', 'strptime', 'time', 'timestamp', 'timetuple', 'timetz', 'today', 'toordinal', 'tzinfo', 'tzname', 'utcfromtimestamp', 'utcnow', 'utcoffset', 'utctimetuple', 'weekday', 'year']


In [ ]:
print(datetime.today)
print(datetime.today())

<built-in method today of type object at 0x7fc80471ad40>
2023-03-08 16:01:46.761279


### module names
import errors often come up when you named your own modules the same as standard library modules

### variable names
the same applies for variable names if you accidentally name a varibale the as a function that function 
is not available anymore

### use `from ... import ...`
- import * is not recommended just import what you need
- some modules even have functions with the same name and one overwrites the other when you imported the entire module

### learn to package your code 
and install it in the current environment

### python is compiled
-  .pyc files or __pyache__ are compiled python code
- but python is also an interpreted language
- python is compiled to bytecode which is then run by the interpreter

### adhere to pep8 
- pep8 is a styleguide
- pro's use it
- pep8 is a styleguide and pro's use it
https://peps.python.org/pep-0008/#introduction

# **SOLID** Principles<br>
- Single Responsibility Principle<br>
- Open/Closed Principle<br>
- Liskov Substitution Principle<br>
- Interface Segregation Principle<br>
- Dependency Inversion<br>

**Single Responsibility Principle** <br>
A class should have one, and only one, reason to change.

**Open/Closed Principle**<br>
“Software entities (classes, modules, functions, etc.) should be open for extension, but closed for modification.”

**Liskov Substitution Principle**<br>
The principle defines that objects of a superclass shall be replaceable with objects of its subclasses without breaking the application. That requires the objects of your subclasses to behave in the same way as the objects of your superclass.

**Interface Segregation Principle**<br>
"Clients should not be forced to depend upon interfaces that they do not use."

**Dependency Inversion Principle** consists of two parts:
1. High-level modules should not depend on low-level modules. Both should depend on abstractions.
2. Abstractions should not depend on details. Details should depend on abstractions.



In [ ]:
import logging
import socket
import subprocess
import time
from collections import namedtuple

import numpy as np


def manual_str_formatting(name, subscribers):
    if subscribers > 100000:
        print("Wow " + name + "! you have " + str(subscribers) + " subscribers!")
    else:
        print("Lol " + name + " that's not many subs")

    # better
    if subscribers > 100000:
        print(f"Wow {name}! you have {subscribers} subscribers!")
    else:
        print(f"Lol {name} that's not many subs")


def manually_calling_close_on_a_file(filename):
    f = open(filename, "w")
    f.write("hello!\n")
    f.close()

    with open(filename, "w") as f:
        f.write("hello!\n")
    # close automatic, even if exception

def finally_instead_of_context_manager(host, port):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        s.connect((host, port))
        s.sendall(b'Hello, world')
    finally:
        s.close()

    # close even if exception
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((host, port))
        s.sendall(b'Hello, world')


def bare_except():
    while True:
        try:
            s = input("Input a number: ")
            x = int(s)
            break
        except:  # oops! can't CTRL-C to exit
            print("Not a number, try again")

    while True:
        try:
            s = input("Input a number: ")
            x = int(s)
            break
        except Exception:  # still better to use ValueError
            print("Not a number, try again")


def caret_and_exponentiation(x, p):
    y = x ^ p  # bitwise xor of x and p, not exponentiation
    y = x ** p


def mutable_default_arguments():
    def append(n, l=[]):
        l.append(n)
        return l

    l1 = append(0)  # [0]
    l2 = append(1)  # [0, 1]

    def append(n, l=None):
        if l is None:
            l = []
        l.append(n)
        return l

    l1 = append(0)  # [0]
    l2 = append(1)  # [1]

def never_using_comprehensions():
    squares = {}
    for i in range(10):
        squares[i] = i * i

    # same
    odd_squares = {i: i * i for i in range(10)}

def always_using_comprehensions(a, b, n):
    """matrix product of a, b of length n x n"""
    c = [
        sum(a[n * i + k] * b[n * k + j] for k in range(n))
        for i in range(n)
        for j in range(n)
    ]

    c = []
    for i in range(n):
        for j in range(n):
            ij_entry = sum(a[n * i + k] * b[n * k + j] for k in range(n))
            c.append(ij_entry)

    return c

def checking_type_equality():
    Point = namedtuple('Point', ['x', 'y'])
    p = Point(1, 2)

    if type(p) == tuple:
        print("it's a tuple")
    else:
        print("it's not a tuple")

    # probably meant to check if is instance of tuple
    if isinstance(p, tuple):
        print("it's a tuple")
    else:
        print("it's not a tuple")


def equality_for_singletons(x):
    if x == None:
        pass

    if x == True:
        pass

    if x == False:
        pass

    # better
    if x is None:
        pass

    if x is True:
        pass

    if x is False:
        pass

def checking_bool_or_len(x):
    if bool(x):
        pass

    if len(x) != 0:
        pass

    # usually equivalent to
    if x:
        pass

def range_len_pattern():
    a = [1, 2, 3]
    for i in range(len(a)):
        v = a[i]
        ...

    # instead
    for v in a:
        ...

    # or if you wanted the index
    for i, v in enumerate(a):
        ...

    # using i to sync between two things?
    b = [4, 5, 6]
    for i in range(len(b)):
        av = a[i]
        bv = b[i]
        ...

    # instead use zip
    for av, bv in zip(a, b):
        ...

def for_key_in_dict_keys():
    d = {"a": 1, "b": 2, "c": 3}
    for key in d.keys():
        ...

    # that's the default
    for key in d:
        ...

    # or if you meant to make a copy of keys
    for key in list():
        ...

def not_using_dict_items():
    d = {"a": 1, "b": 2, "c": 3}
    for key in d:
        val = d[key]
        ...

    for key, val in d.items():
        ...

def tuple_unpacking():
    x = 0
    y = 1

    tmp = x
    x = y
    y = tmp

    x, y = 0, 1
    x, y = y, x

    mytuple = 1, 2
    x = mytuple[0]
    y = mytuple[1]

    x, y = mytuple


def index_counter_variable():
    l = [1, 2, 3]

    i = 0
    for x in l:
        ...
        i += 1

    for i, x in enumerate(l):
        ...

def timing_with_time():
    start = time.time()
    time.sleep(1)
    end = time.time()
    print(end - start)

    # more accurate
    start = time.perf_counter()
    time.sleep(1)
    end = time.perf_counter()
    print(end - start)



def print_vs_logging():
    print("debug info")
    print("just some info")
    print("bad error")

    # versus
    # in main
    level = logging.DEBUG
    fmt = '[%(levelname)s] %(asctime)s - %(message)s'
    logging.basicConfig(level=level, format=fmt)

    # wherever
    logging.debug("debug info")
    logging.info("just some info")
    logging.error("uh oh :(")

def subprocess_with_shell_true():
    subprocess.run(["ls -l"], capture_output=True, shell=True)

    subprocess.run(["ls", "-l"], capture_output=True)

def not_using_numpy_pandas():
    x = list(range(100))
    y = list(range(100))
    s = [a + b for a, b in zip(x, y)]

    # better (faster)
    x = np.arange(100)
    y = np.arange(100)
    s = x + y


def not_following_pep8():
    x = (1, 2)
    y=5
    l = [1,2,3]

    def func(x=5):
        ...


def python2_thinking():
    x = 10000000000000000000
    print(x in range(2 * x))  # ranges are lazy, this will be fast

    d = {"a": 1, "b": 2, "c": 3}
    keys = d.keys()
    del d["a"]
    print("a" in keys)  # keys is a "view", not a copy